# Plan & Execute Agent Tutorial

This notebook demonstrates the **Plan & Execute** pattern for complex, multi-step tasks that benefit from explicit planning before execution.

## What is Plan & Execute?

Plan & Execute separates **planning** from **execution**:

```
Complex Task → PLAN (create subtasks) → EXECUTE (run each) → SYNTHESIZE (combine results)
```

### The Pattern:

1. **PLAN**: Analyze the task and create a structured plan with subtasks and dependencies
2. **EXECUTE**: Run each subtask (can use ReAct for complex subtasks)
3. **SYNTHESIZE**: Combine all results into a comprehensive final answer
4. **REPLAN** (optional): If subtasks fail, adjust the plan

### When to Use Plan & Execute vs ReAct

| Use Case | ReAct | Plan & Execute |
|----------|-------|----------------|
| "What's AAPL's price?" | ✅ | |
| "Compare 5 stocks and create a report" | | ✅ |
| Quick lookups | ✅ | |
| Multi-step research tasks | | ✅ |
| Unknown number of steps | ✅ | |
| Clear subtask dependencies | | ✅ |

## Setup

Install dependencies and configure API key.

In [ ]:
# Install miiflow-llm with example dependencies
!pip install -q miiflow-llm[examples] nest_asyncio

# Enable nested async for environments that need it (e.g., some Jupyter kernels)
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import os
from getpass import getpass

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

print("API key configured!")

## Define Tools

We'll use the same stock tools plus some additional tools for analysis and report generation.

In [ ]:
import yfinance as yf
from datetime import datetime
from miiflow_llm.core.tools import tool


@tool("get_stock_quote", "Get real-time stock quote for a symbol")
def get_stock_quote(symbol: str) -> str:
    """Fetch current stock price and metrics."""
    try:
        ticker = yf.Ticker(symbol.upper())
        info = ticker.info
        
        if not info or "regularMarketPrice" not in info:
            return f"Unable to fetch data for '{symbol}'"
        
        price = info.get("regularMarketPrice", "N/A")
        prev = info.get("regularMarketPreviousClose", price)
        change = price - prev if isinstance(price, (int, float)) and isinstance(prev, (int, float)) else 0
        change_pct = (change / prev * 100) if prev else 0
        
        cap = info.get("marketCap", 0)
        cap_str = f"${cap/1e12:.2f}T" if cap >= 1e12 else f"${cap/1e9:.2f}B" if cap >= 1e9 else f"${cap:,.0f}"
        
        # Format P/E ratio
        pe_ratio = info.get('trailingPE')
        pe_str = f"{pe_ratio:.2f}" if isinstance(pe_ratio, (int, float)) else "N/A"
        
        # Format 52-week range
        week_low = info.get('fiftyTwoWeekLow', 0)
        week_high = info.get('fiftyTwoWeekHigh', 0)
        week_low_str = f"${week_low:.2f}" if isinstance(week_low, (int, float)) else "N/A"
        week_high_str = f"${week_high:.2f}" if isinstance(week_high, (int, float)) else "N/A"
        
        return f"""{info.get('shortName', symbol)} ({symbol.upper()}):
Price: ${price:.2f} ({change:+.2f}, {change_pct:+.2f}%)
Market Cap: {cap_str}
P/E: {pe_str}
52W Range: {week_low_str} - {week_high_str}"""
    except Exception as e:
        return f"Error: {str(e)}"


@tool("get_stock_history", "Get historical price data")
def get_stock_history(symbol: str, period: str = "1mo") -> str:
    """Fetch historical data. Period: 1d, 5d, 1mo, 3mo, 6mo, 1y"""
    try:
        hist = yf.Ticker(symbol.upper()).history(period=period)
        if hist.empty:
            return f"No data for {symbol}"
        
        start, end = hist['Close'].iloc[0], hist['Close'].iloc[-1]
        change = end - start
        change_pct = (change / start) * 100
        
        return f"""{symbol.upper()} ({period}):
Start: ${start:.2f} → End: ${end:.2f}
Change: {change:+.2f} ({change_pct:+.2f}%)
High: ${hist['High'].max():.2f}, Low: ${hist['Low'].min():.2f}"""
    except Exception as e:
        return f"Error: {str(e)}"


@tool("get_company_info", "Get company profile information")
def get_company_info(symbol: str) -> str:
    """Fetch company profile."""
    try:
        info = yf.Ticker(symbol.upper()).info
        employees = info.get('fullTimeEmployees', 'N/A')
        employees_str = f"{employees:,}" if isinstance(employees, int) else str(employees)
        return f"""{info.get('longName', symbol)} ({symbol.upper()}):
Sector: {info.get('sector', 'N/A')}
Industry: {info.get('industry', 'N/A')}
Employees: {employees_str}
Summary: {info.get('longBusinessSummary', 'N/A')[:300]}..."""
    except Exception as e:
        return f"Error: {str(e)}"


@tool("analyze_stocks", "Analyze and compare stock data")
def analyze_stocks(data: str, analysis_type: str = "comparison") -> str:
    """Analyze stock data. Types: comparison, trend, risk"""
    return f"""Stock Analysis ({analysis_type}):
Based on the provided data:
- Market trends show mixed signals
- Volatility is within normal ranges
- Sector performance varies

Key observations from data:
{data[:200]}...

Note: This is simulated analysis for demonstration."""


@tool("generate_report", "Generate a formatted investment report")
def generate_report(title: str, content: str) -> str:
    """Generate a formatted report."""
    return f"""
{'='*50}
{title.upper()}
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M')}
{'='*50}

{content}

{'='*50}
DISCLAIMER: For demonstration purposes only.
{'='*50}
"""


print("Tools defined: get_stock_quote, get_stock_history, get_company_info, analyze_stocks, generate_report")

## Example 1: Basic Plan & Execute

Let's start with a task that benefits from planning: analyzing multiple stocks.

The agent will:
1. Create a plan with subtasks
2. Execute each subtask
3. Combine results into a final answer

In [ ]:
from miiflow_llm import LLMClient, Agent, AgentType

# Create client
client = LLMClient.create("openai", model="gpt-5.1")

# Create Plan & Execute agent
agent = Agent(
    client,
    agent_type=AgentType.PLAN_AND_EXECUTE,  # Key difference from ReAct!
    max_iterations=15,  # More iterations for complex tasks
    system_prompt="You are a financial analyst. Create structured plans for research tasks.",
)

# Add tools
agent.add_tool(get_stock_quote)
agent.add_tool(get_stock_history)
agent.add_tool(get_company_info)
agent.add_tool(analyze_stocks)
agent.add_tool(generate_report)

print("Plan & Execute agent created!")

In [ ]:
# A task that benefits from planning
task = """
Research and compare these tech stocks: AAPL, MSFT, and GOOGL.

For each stock:
1. Get the current price and metrics
2. Check the monthly performance

Then provide a comparison summary.
"""

print("Task:", task)
print("\nExecuting with Plan & Execute...\n")

result = await agent.run(task)
print(result.data)

## Example 2: Watch the Planning Process

Let's stream the agent's execution to see:
1. The planning phase (creating subtasks)
2. Each subtask being executed
3. The final synthesis

In [ ]:
from miiflow_llm import RunContext
from miiflow_llm.core.react import PlanExecuteEventType

task = """
Compare Tesla (TSLA) and Ford (F):
1. Get current prices for both
2. Get company profiles for both
3. Provide a comparison
"""

print("Task:", task)
print("\n" + "="*50)

# Create a context for streaming
context = RunContext(deps=None, messages=[])

async for event in agent.stream(task, context):
    event_type = event.event_type
    
    if event_type == PlanExecuteEventType.PLANNING_START:
        print("\n[PHASE 1: PLANNING]")
        print("Creating execution plan...\n")
    
    elif event_type == PlanExecuteEventType.PLANNING_COMPLETE:
        subtask_count = event.data.get("subtask_count", 0)
        print(f"\nPlan created with {subtask_count} subtasks!")
        print("\n[PHASE 2: EXECUTION]")
    
    elif event_type == PlanExecuteEventType.SUBTASK_START:
        desc = event.data.get("description", "")[:60]
        subtask_id = event.data.get("subtask_id", "?")
        print(f"\n  Subtask {subtask_id}: {desc}...")
    
    elif event_type == PlanExecuteEventType.SUBTASK_COMPLETE:
        result_preview = str(event.data.get("result", ""))[:80]
        print(f"  ✓ Done: {result_preview}...")
    
    elif event_type == PlanExecuteEventType.FINAL_ANSWER:
        print("\n" + "="*50)
        print("[PHASE 3: FINAL ANSWER]")
        print("="*50)
        print(event.data.get("answer", ""))

## Example 3: Comprehensive Research Report

Plan & Execute excels at complex research tasks. Let's create a full investment report.

In [ ]:
task = """
Create a comprehensive investment report on the "Magnificent 7" tech stocks.
Focus on: AAPL, MSFT, and NVDA (for time efficiency).

Your report should include:
1. Current price and market cap for each stock
2. Monthly performance trends
3. Brief company profiles
4. A final comparison and investment considerations

Generate a professional formatted report at the end.
"""

print("Creating comprehensive report...")
print("This may take a moment as the agent plans and executes multiple subtasks.\n")

result = await agent.run(task)

print("="*60)
print("GENERATED REPORT")
print("="*60)
print(result.data)

## Example 4: Single Stock Deep Dive

Plan & Execute is also useful for thorough single-entity research with multiple aspects to investigate.

In [ ]:
task = """
Perform a deep dive analysis on NVIDIA (NVDA):

1. Get current stock price and key metrics
2. Analyze 3-month price history
3. Review company profile and business
4. Provide investment considerations
"""

result = await agent.run(task)
print(result.data)

## Understanding the Plan Structure

When Plan & Execute creates a plan, it structures subtasks with:
- **Description**: What the subtask does
- **Dependencies**: Which subtasks must complete first
- **Required tools**: Which tools will be needed

Example plan structure:
```
Subtask 1: Get AAPL price (no dependencies)
Subtask 2: Get MSFT price (no dependencies)
Subtask 3: Get GOOGL price (no dependencies)
Subtask 4: Compare stocks (depends on 1, 2, 3)
Subtask 5: Generate report (depends on 4)
```

Independent subtasks (1, 2, 3) can potentially run in parallel, while dependent subtasks wait for their prerequisites.

## Try It Yourself!

Create your own complex research task:

In [ ]:
# Try your own complex task!
your_task = """
Research the semiconductor industry:
1. Get data on NVDA, AMD, and INTC
2. Compare their market caps and P/E ratios
3. Summarize the competitive landscape
"""

result = await agent.run(your_task)
print(result.data)

## Key Takeaways

### Plan & Execute Pattern
1. **PLAN**: Break down complex tasks into subtasks with dependencies
2. **EXECUTE**: Run subtasks in order, respecting dependencies
3. **SYNTHESIZE**: Combine results into comprehensive answer

### When to Use
- Multi-entity research (comparing multiple stocks)
- Tasks with clear phases (gather → analyze → report)
- Complex workflows where organization helps
- When you want visibility into the execution plan

### Key Parameters
- `agent_type=AgentType.PLAN_AND_EXECUTE` - enables planning mode
- `max_iterations=15+` - complex tasks need more iterations

### vs ReAct
- **ReAct**: Flexible, adaptive, lower overhead → simple queries
- **Plan & Execute**: Structured, organized, visible planning → complex research

## Summary

| Pattern | Best For | Overhead | Structure |
|---------|----------|----------|----------|
| ReAct | Quick lookups, simple queries | Low | Flexible |
| Plan & Execute | Multi-step research, reports | Higher | Organized |